In [10]:
import pandas as pd

#Importo le informazioni riguardanti le qualità delle tasks 
tasks = pd.read_csv("../Dataset 2.0/Tasks.csv")
tasks = tasks.drop(0, axis = 0)
tasks = tasks.drop("Deadline", axis = 1)


#Importo i coefficienti di regressione di ogni agente
coeffs = pd.read_csv("../Coefficienti di Regressione/coeffs.csv")

decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Selezionare il Session ID ed il Round voluti
#Assicurarsi di selezionare lo stesso round in "RegressionModel.py" e in "OptimizationModel.py"
decisions = decisions[decisions["Session ID"] == "bfce2734-f71a-4ce9-8754-663ce6b25b64"]
decisions = decisions[decisions["Round"] == 2]
decisions = decisions.sort_values("Worker Agent ID")
decisions

,ID,Session ID,Round,Worker Agent ID,Worker Agent Reputation
174911,529209,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,1,0.500000
174912,529213,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,2,0.500000
174913,529216,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,3,0.500000
174916,529223,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,4,0.697631
174918,529230,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,5,0.831661
174920,529236,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,6,0.831661
174923,529243,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,7,0.798797
174926,529251,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,8,0.798797
174930,529260,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,9,0.798797
174934,529269,bfce2734-f71a-4ce9-8754-663ce6b25b64,2,10,0.749373


In [11]:
from mip import *

m = Model()

#DATI


#Identifier del Worker Agent
WAID = list(decisions["Worker Agent ID"])


#Peso dell'Identifier del WA
wWAID = coeffs.iloc[91].values.tolist()

if (WAID[9] == 10):
    wWAID = wWAID[0:10]

else:
    wWAID = wWAID[10:20]

#Identifier del Round
dID = list(decisions["ID"])


#Peso dell'Identifier del round
wdID = coeffs.iloc[0].values.tolist()

if (WAID[9] == 10):
    wdID = wdID[0:10]

else:
    wdID = wdID[10:20]


#Value di ogni task
Value = []
for i in tasks["Value"]:
    Value.append(int(i))
    

#Peso del Value (coeffs 1, 4, 7, ..., 88)
wValue = []

j = 1
for k in range(30):
    wValue.append(coeffs.iloc[j].values.tolist())
    j = j + 3
    
if (WAID[9] == 10):
    for h in range(30):
        wValue[h] = wValue[h][0:10]

else:
    for h in range(30):
        wValue[h] = wValue[h][10:20]


#Complessità di ogni task
Difficulty = []
for i in tasks["Difficulty"]:
    Difficulty.append(float(i))


#Peso della Complessità (coeffs 2, 5, 8, ..., 89)
wDifficulty = []

j = 2
for k in range(30):
    wDifficulty.append(coeffs.iloc[j].values.tolist())
    j = j + 3
    
if (WAID[9] == 10):
    for h in range(30):
        wDifficulty[h] = wDifficulty[h][0:10]

else:
    for h in range(30):
        wDifficulty[h] = wDifficulty[h][10:20]


#Stamina consumata da ogni task
Effort = []
for i in tasks["Effort Required"]:
    Effort.append(int(i))
    

#Peso della Stamina consumata (coeffs 3, 6, 9, ..., 90)
wEffort = []

j = 3
for k in range(30):
    wEffort.append(coeffs.iloc[j].values.tolist())
    j = j + 3

if (WAID[9] == 10):
    for h in range(30):
        wEffort[h] = wEffort[h][0:10]

else:
    for h in range:
        wEffort[h] = wEffort[h][10:20]
    
#Reputazione del WA prima del round
Rep = list(decisions["Worker Agent Reputation"])


#Peso della reputation
wRep = coeffs.iloc[92].values.tolist()

if (WAID[9] == 10):
    wRep = wRep[0:10]

else:
    wRep = wRep[10:20]

#creo i range utili per la creazione di variabili
(A, T) = (range(len(WAID)), range(len(Value)))


#VARIABILI

#x[a, t] vale 1 se la task t è assegnata al WA a, 0 altrimenti
x = [[m.add_var(var_type=BINARY) for t in T] for a in A]

#y(a) contiene la reputation del WA a fine round
y = [m.add_var(var_type=CONTINUOUS) for a in A]


#VINCOLI

#Vincolo che impone che ogni task venga assegnata ad uno ed un solo WA
for t in T:
    m += xsum(x[a][t] for a in A) == 1


#OBIETTIVO
#Massimizzare la sommatoria delle reputation dei worker agent
m.objective = maximize(xsum(dID[a]*wdID[a] for a in A) + xsum(wValue[t][a]*Value[t]*x[a][t] + wDifficulty[t][a]*Difficulty[t]*x[a][t] + wEffort[t][a]*Effort[t]*x[a][t] for t in T for a in A) + xsum(wRep[a]*Rep[a] + wWAID[a]*WAID[a] for a in A))

m.write("model.lp")
m.optimize()

print(f"Totale: {m.objective_value}")

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables - nothing to do
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Starting solution of the Linear programming relaxation problem using Dual Simplex

Coin0506I Presolve 0 (-30) rows, 0 (-310) columns and 0 (-300) elements
Clp0000I Optimal - objective value 0.8284731
Coin0511I After Postsolve, objective 0.8284731, infeasibilities - dual 1.8310688 (138), primal 0 (0)
Coin0512I Presolved model was optimal, full model needs cleaning up
Clp0006I 0  Obj 0.8284731 Dual inf 1.8309446 (138)
Clp0000I Optimal - objective value 0.8284731
Clp0032I Optimal objective 0.8284731027 - 26 iterations time 0.002, Presolve 0.00

Starting MIP optimization
Totale: 5.535981945370644


In [13]:
tot = 0
for a in A:
    for t in T:
        if x[a][t].x == 1:
            tot += 1
            print(f"Agente {a + 1}, Task {t +1}")


Agente 1, Task 23
Agente 2, Task 20
Agente 2, Task 22
Agente 3, Task 26
Agente 4, Task 2
Agente 4, Task 11
Agente 4, Task 30
Agente 5, Task 9
Agente 5, Task 17
Agente 5, Task 18
Agente 5, Task 21
Agente 5, Task 25
Agente 5, Task 29
Agente 6, Task 5
Agente 6, Task 6
Agente 6, Task 8
Agente 6, Task 15
Agente 6, Task 16
Agente 6, Task 19
Agente 6, Task 24
Agente 8, Task 4
Agente 8, Task 7
Agente 8, Task 10
Agente 8, Task 14
Agente 9, Task 1
Agente 9, Task 27
Agente 9, Task 28
Agente 10, Task 3
Agente 10, Task 12
Agente 10, Task 13


In [67]:
decisions2 = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Da "Decisions.csv" seleziono solo le tuple riguardanti il round successivo
decisions2 = decisions2[decisions2["Round"] == 2]

#Estraggo "Session ID", "Worker Agent ID" e "Worker Agent Reputation"
decisions2 = decisions2.iloc[:, [1, 3, 4]]
decisions2 = decisions2[decisions2["Session ID"] == "02647ffb-6535-4f65-adc5-bc7e0e5bbc66"]
tmp1 = pd.merge(decisions, decisions2, left_on = "Worker Agent ID", right_on = "Worker Agent ID")

tmp1 = tmp1.iloc[:, [0, 1, 2, 3, 4, 6]]

totale = 0
for i in range(len(tmp1)):
    totale += tmp1.iloc[i, 5]
totale

6.763686228852361